# LlamaIndex 워크플로우 절차

**작성일**: 2025년 8월 26일

최종 정리된 절차
```
파일 준비: .txt, .pdf, .doc 등 파일을 디렉토리에 준비.
파일 읽기: SimpleDirectoryReader로 파일을 읽어 Document 객체 생성.
텍스트 분할 및 노드 생성: SimpleNodeParser, SentenceSplitter 등으로 문서를 청크 단위로 분할하여 Node 객체 생성.
인덱스 생성: VectorStoreIndex로 노드 또는 문서를 벡터화하고 인덱스 생성.
쿼리 엔진 생성: index.as_query_engine()으로 검색 및 답변 생성 준비.
쿼리 실행: query_engine.query()로 질문 처리.
응답 출력: response.response로 답변 출력, response.source_nodes로 검색된 원본 확인.
```


### 1. 파일 준비
 .txt, .pdf, .doc 등 다양한 형식의 파일을 특정 디렉토리(예: data 또는 sample_docs)에 준비합니다.
 예: SimpleDirectoryReader("data") 또는 SimpleDirectoryReader("sample_docs")로 디렉토리를 지정.

### 2. 파일 읽기
 SimpleDirectoryReader를 사용해 파일을 읽고 Document 객체로 변환합니다.
 recursive=True로 설정하면 하위 폴더도 포함하며, required_exts로 특정 파일 확장자(예: .txt, .pdf)만 로드 가능.
 예:
  documents = SimpleDirectoryReader("data", recursive=True, required_exts=[".txt", ".pdf"]).load_data()

### 3. 텍스트 분할 및 노드 생성
 문서를 작은 텍스트 청크로 나누고 Node 객체로 변환합니다. 검색과 임베딩을 효율적으로 처리하기 위해 필요.
 사용 가능한 파서:
   SimpleNodeParser: 고정된 크기(예: chunk_size=80)로 분할.
   TokenTextSplitter: 토큰 단위로 분할.
   SentenceSplitter: 문장 단위로 분할.
   SemanticSplitterNodeParser: 의미 단위로 분할(임베딩 모델 필요).
 예:
  parser = SimpleNodeParser(chunk_size=80, chunk_overlap=0)
  nodes = parser.get_nodes_from_documents(documents)
 참고: 이 단계에서는 텍스트 분할만 수행되며, 벡터 임베딩은 이루어지지 않습니다.

### 4. 인덱스 생성
 VectorStoreIndex를 사용해 분할된 노드(또는 문서)를 벡터화하고 인덱스를 생성.
 텍스트 청크는 임베딩 모델(기본적으로 OpenAI text-embedding-ada-002)로 벡터화되어 인덱스에 저장.
 from_documents를 사용하면 문서를 직접 전달해 내부적으로 분할 및 임베딩 수행. 또는 nodes를 전달해 미리 분할된 노드 사용.
 예:
  index = VectorStoreIndex.from_documents(documents)
  또는
  index = VectorStoreIndex(nodes)

### 5. 쿼리 엔진 생성
 인덱스를 기반으로 쿼리 엔진을 생성하여 검색 및 답변 생성 준비.
 similarity_top_k로 검색 결과 수 조정, node_postprocessors로 후처리 설정(예: 유사도 임계값) 가능.
 예:
  query_engine = index.as_query_engine(similarity_top_k=3)

### 6. 쿼리 실행
 query_engine.query()를 사용해 사용자의 질문(쿼리)을 처리.
 쿼리는 인덱스에서 유사한 노드를 검색한 후, 지정된 LLM(예: gpt-4o-mini)을 사용해 답변 생성.
 예:
  response = query_engine.query("이 영화의 반전은 무엇인가요?")

### 7. 응답 출력
 쿼리 결과는 response 객체에 저장되며, response.response로 최종 답변 텍스트를 얻거나, response.source_nodes로 검색된 원본 노드 확인 가능.
 예:
  print(response.response)
  for node in response.source_nodes:
      print(node.text)

### 추가 세부 사항
 임베딩 모델: 인덱스 생성 시 텍스트 청크는 자동으로 벡터화. 기본적으로 OpenAI 임베딩 API 사용, HuggingFaceEmbedding 같은 로컬 모델로 변경 가능.
  from llama_index.embeddings.huggingface import HuggingFaceEmbedding
  Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
 LLM 설정: 쿼리 엔진에서 답변 생성 시 사용되는 LLM은 Settings.llm 또는 llm 매개변수로 지정. 예: gpt-4o-mini 사용, 시스템 프롬프트로 "반드시 한국어로 답변하세요" 설정 가능.
 벡터 저장소: 기본적으로 인메모리 벡터 저장소 사용, ChromaVectorStore 같은 외부 벡터 DB로 영구 저장 가능.

### 정리
 3번(텍스트 분할): 벡터 임베딩이 아닌 텍스트 분할만 수행.
 4번(인덱스 생성): 텍스트 청크가 벡터로 변환되어 인덱스에 저장.
 7번(응답 출력): response.response 외에도 response.source_nodes로 검색된 원본 텍스트 확인 가능.




LlamaIndex 워크플로우 절차
작성일: 2025년 8월 26일

1. 파일 준비
 .txt, .pdf, .doc 등 다양한 형식의 파일을 특정 디렉토리(예: data 또는 sample_docs)에 준비합니다.
 예: SimpleDirectoryReader("data") 또는 SimpleDirectoryReader("sample_docs")로 디렉토리를 지정.

2. 파일 읽기
 SimpleDirectoryReader를 사용해 파일을 읽고 Document 객체로 변환합니다.
 recursive=True로 설정하면 하위 폴더도 포함하며, required_exts로 특정 파일 확장자(예: .txt, .pdf)만 로드 가능.
 예:
  documents = SimpleDirectoryReader("data", recursive=True, required_exts=[".txt", ".pdf"]).load_data()

3. 텍스트 분할 및 노드 생성
 문서를 작은 텍스트 청크로 나누고 Node 객체로 변환합니다. 검색과 임베딩을 효율적으로 처리하기 위해 필요.
 사용 가능한 파서:
   SimpleNodeParser: 고정된 크기(예: chunk_size=80)로 분할.
   TokenTextSplitter: 토큰 단위로 분할.
   SentenceSplitter: 문장 단위로 분할.
   SemanticSplitterNodeParser: 의미 단위로 분할(임베딩 모델 필요).
 예:
  parser = SimpleNodeParser(chunk_size=80, chunk_overlap=0)
  nodes = parser.get_nodes_from_documents(documents)
 이 단계에서는 텍스트 분할만 수행되며, 벡터 임베딩은 이루어지지 않음.

4. 인덱스 생성
 VectorStoreIndex를 사용해 분할된 노드(또는 문서)를 벡터화하고 인덱스를 생성.
 텍스트 청크는 임베딩 모델(기본적으로 OpenAI textembeddingada002)로 벡터화되어 인덱스에 저장.
 from_documents를 사용하면 문서를 직접 전달해 내부적으로 분할 및 임베딩 수행. 또는 nodes를 전달해 미리 분할된 노드 사용.
 예:
  index = VectorStoreIndex.from_documents(documents)
  또는
  index = VectorStoreIndex(nodes)

5. 쿼리 엔진 생성
 인덱스를 기반으로 쿼리 엔진을 생성하여 검색 및 답변 생성 준비.
 similarity_top_k로 검색 결과 수 조정, node_postprocessors로 후처리 설정(예: 유사도 임계값) 가능.
 예:
  query_engine = index.as_query_engine(similarity_top_k=3)

6. 쿼리 실행
 query_engine.query()를 사용해 사용자의 질문(쿼리)을 처리.
 쿼리는 인덱스에서 유사한 노드를 검색한 후, 지정된 LLM(예: gpt4omini)을 사용해 답변 생성.
 예:
  response = query_engine.query("이 영화의 반전은 무엇인가요?")

7. 응답 출력
 쿼리 결과는 response 객체에 저장되며, response.response로 최종 답변 텍스트를 얻거나, response.source_nodes로 검색된 원본 노드 확인 가능.
 예:
  print(response.response)
  for node in response.source_nodes:
      print(node.text)

추가 세부 사항
 임베딩 모델: 인덱스 생성 시 텍스트 청크는 자동으로 벡터화. 기본적으로 OpenAI 임베딩 API 사용, HuggingFaceEmbedding 같은 로컬 모델로 변경 가능.
  예:
  from llama_index.embeddings.huggingface import HuggingFaceEmbedding
  Settings.embed_model = HuggingFaceEmbedding(model_name="sentencetransformers/allMiniLML6v2")
 LLM 설정: 쿼리 엔진에서 답변 생성 시 사용되는 LLM은 Settings.llm 또는 llm 매개변수로 지정. 예: gpt4omini 사용, 시스템 프롬프트로 "반드시 한국어로 답변하세요" 설정 가능.
 벡터 저장소: 기본적으로 인메모리 벡터 저장소 사용, ChromaVectorStore 같은 외부 벡터 DB로 영구 저장 가능.

정리
 3번(텍스트 분할): 벡터 임베딩이 아닌 텍스트 분할만 수행.
 4번(인덱스 생성): 텍스트 청크가 벡터로 변환되어 인덱스에 저장.
 7번(응답 출력): response.response 외에도 response.source_nodes로 검색된 원본 텍스트 확인 가능.

In [ ]:
pip install llama-index
pip install ipykernel
pip install python-dotenv

In [ ]:
from dotenv  import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
api_key

In [ ]:
# !pip install llama-index
!llamaindex-cli download-llamadataset PaulGrahamEssayDataset --download-dir ./data

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

2025-08-26 09:48:10,404 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [7]:
query_engine = index.as_query_engine()
response = query_engine.query("작가의 청소년 시절은 어떠했나요?")
print(response)

2025-08-26 09:48:13,157 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 09:48:16,664 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


작가는 청소년 시절에 회화를 감상하는 것을 좋아했지만, 그것을 스스로 그릴 수 있는 능력에 대해 의심을 품었습니다. 그는 회화를 만들 수 있는 사람들을 다른 종류의 존재로 여겼으며, 회화를 만들 수 있는 능력에 대해 거의 기적적으로 생각했습니다. 이후 하버드에서 미술 수업을 듣기 시작하면서 예술가로서의 길을 모색하기 시작했습니다.


In [8]:
print(response)

작가는 청소년 시절에 회화를 감상하는 것을 좋아했지만, 그것을 스스로 그릴 수 있는 능력에 대해 의심을 품었습니다. 그는 회화를 만들 수 있는 사람들을 다른 종류의 존재로 여겼으며, 회화를 만들 수 있는 능력에 대해 거의 기적적으로 생각했습니다. 이후 하버드에서 미술 수업을 듣기 시작하면서 예술가로서의 길을 모색하기 시작했습니다.


In [9]:
response.response

'작가는 청소년 시절에 회화를 감상하는 것을 좋아했지만, 그것을 스스로 그릴 수 있는 능력에 대해 의심을 품었습니다. 그는 회화를 만들 수 있는 사람들을 다른 종류의 존재로 여겼으며, 회화를 만들 수 있는 능력에 대해 거의 기적적으로 생각했습니다. 이후 하버드에서 미술 수업을 듣기 시작하면서 예술가로서의 길을 모색하기 시작했습니다.'

VectorStoreIndex.from_documents(documents)를 호출할 때, LlamaIndex는 내부적으로 문서의 텍스트를 자동으로 벡터화하여 저장합니다. 따라서 별도로 벡터로 변환하는 과정을 수동으로 수행할 필요는 없습니다. 이 과정은 다음과 같이 진행됩니다:

텍스트 분할: SimpleDirectoryReader로 로드된 문서(documents)는 먼저 내부적으로 텍스트 청크(chunk)로 분할됩니다. 이는 SimpleNodeParser 또는 설정된 다른 파서(예: SentenceSplitter)를 통해 수행되며, 기본적으로 적절한 크기의 텍스트 조각으로 나뉩니다.

임베딩 생성: 분할된 각 텍스트 청크는 LlamaIndex에서 설정된 임베딩 모델(기본적으로 OpenAI의 text-embedding-ada-002와 같은 모델)을 사용하여 벡터로 변환됩니다. 이 벡터는 텍스트의 의미를 숫자 형태로 표현한 것으로, 검색 및 유사도 계산에 사용됩니다.
LlamaIndex는 기본적으로 OpenAI의 text-embedding-ada-002 모델을 사용하지만, Settings.embed_model을 통해 다른 모델(예: HuggingFace 모델)을 지정할 수 있습니다.

벡터 저장: 변환된 벡터는 VectorStoreIndex에 저장됩니다. 이 인덱스는 벡터와 원본 텍스트, 그리고 메타데이터를 함께 저장하여 나중에 쿼리 시 효율적으로 검색할 수 있도록 합니다.

# 제 2장

In [ ]:
#pip install llama-index==0.11.11 -q
# !pip install docx2txt

## 2.2 데이터 로딩

### 2.2.1 데이터 리더

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("sample_docs").load_data()

2025-08-26 14:16:21,827 - WARNING - Ignoring wrong pointing object 6 0 (offset 0)


In [ ]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)

In [5]:
documents = SimpleDirectoryReader(
    "sample_docs", recursive=True).load_data()

2025-08-26 14:16:24,027 - WARNING - Ignoring wrong pointing object 6 0 (offset 0)


In [6]:
len(documents)  # 4개의 문서가 로드됨

5

In [7]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)


Document 1:
This is the content of file 1.
Document 2:
This is the content of file 2.
Document 3:
This is the content of ﬁle 3. 
Document 4:
This is the content of file 4.
Document 5:
This is the content of file 5.


In [8]:
documents = SimpleDirectoryReader(
    "sample_docs",
    required_exts=[".txt", ".pdf"],
    recursive=True).load_data()

2025-08-26 14:16:27,864 - WARNING - Ignoring wrong pointing object 6 0 (offset 0)


In [9]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)

Document 1:
This is the content of file 1.
Document 2:
This is the content of ﬁle 3. 
Document 3:
This is the content of file 4.


In [11]:
# document 리스트의 각 요소에 접근하여 메타데이터를 출력
for i, document in enumerate(documents):
    print(f"Document {i+1} metadata:")
    print(document.metadata)

Document 1 metadata:
{'file_path': 'c:\\Users\\com\\llama\\llama-index\\ch01\\sample_docs\\file1.txt', 'file_name': 'file1.txt', 'file_type': 'text/plain', 'file_size': 30, 'creation_date': '2025-08-26', 'last_modified_date': '2025-08-25'}
Document 2 metadata:
{'page_label': '1', 'file_name': 'file3.pdf', 'file_path': 'c:\\Users\\com\\llama\\llama-index\\ch01\\sample_docs\\file3.pdf', 'file_type': 'application/pdf', 'file_size': 9283, 'creation_date': '2025-08-26', 'last_modified_date': '2025-08-25'}
Document 3 metadata:
{'file_path': 'c:\\Users\\com\\llama\\llama-index\\ch01\\sample_docs\\sub_sample_docs\\file4.txt', 'file_name': 'file4.txt', 'file_type': 'text/plain', 'file_size': 30, 'creation_date': '2025-08-26', 'last_modified_date': '2025-08-25'}


### 2.2.2 데이터 커넥터 
- 데이터베이스 연결
- 커넥터는 driver 와 같다

In [12]:
#!pip install llama-index-readers-database==0.3.0 -q


In [ ]:
#!pip install pymysql
# !pip install cryptography

In [15]:
# MySQL 접속 및 DB/테이블 생성
import pymysql

conn = pymysql.connect(host='localhost', user='root', password='root', port=3306)
cur = conn.cursor()
cur.execute("CREATE DATABASE IF NOT EXISTS test_db;")
cur.execute("USE test_db;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS users1 (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
""")
conn.commit()
cur.close()
conn.close()
print("test_db와 users 테이블 생성 완료")

test_db와 users 테이블 생성 완료


In [19]:
from sqlalchemy import create_engine
from llama_index.readers.database import DatabaseReader

# MySQL 연결 정보 직접 입력
scheme = "mysql+pymysql"
host = "localhost"
password = "user"
port = "3306"
user = "user"
dbname = "test_db"

connection_string = f"{scheme}://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_string)
reader = DatabaseReader(sql_database=engine)

# 데이터 로드
query = "SELECT name, age FROM users"


In [ ]:
documents = reader.load_data(query=query)
documents

In [22]:
# documents 리스트의 각 요소 출력
for idx, doc in enumerate(documents):
    print(f"Document {idx + 1}:")
    print(f"ID {doc.id_}")
    print(f"Row {doc.text}")
    print("-" * 50)


Document 1:
ID e1dde335-247f-434f-b6c0-1a1f396b9b47
Row name: Alice, age: 30
--------------------------------------------------
Document 2:
ID 9343ac81-9b9b-438e-8d09-5a565ddd540a
Row name: Bob, age: 32
--------------------------------------------------
Document 3:
ID ff3cac07-45d3-4983-ad9d-765ac265495e
Row name: Charlie, age: 33
--------------------------------------------------


In [43]:
print(response)

The key themes in this document include advocating for a strategy of making numerous smaller investments rather than a few large ones, supporting younger and more technically-oriented founders over those with MBAs, allowing founders to retain their positions as CEOs, and the value of giving talks as a method for sparking creativity and sharing knowledge.


## 2.3 텍스트 분할

### 2.3.1 문서와 노드

In [45]:
from llama_index.core import Document

# 텍스트 데이터를 기반으로 문서 생성
document_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다."
document = Document(text=document_text)

# 메타데이터 추가
document.metadata = {'author': '영화 해설', 'subject': '기생충 줄거리'}

print(document)

Doc ID: f34ced02-88fd-4d28-908c-0696770d9acf
Text: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는
평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는
오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다.
결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.


<b>Document 객체의 속성 종류</b>

llama_index.core.Document 객체는 텍스트 데이터와 그와 관련된 메타데이터를 저장하는 데 사용됩니다. metadata는 이 Document 객체가 가지고 있는 여러 속성 중 하나이며, 사용자가 직접 추가하거나 수정할 수 있는 부분입니다.

Document 객체의 주요 속성 종류는 다음과 같습니다.

<b>text</b>: Document의 핵심이 되는 본문 텍스트입니다. 위 예시에서는 '영화 '기생충'은...'으로 시작하는 줄거리 내용이 이 속성에 해당합니다.

<b>metadata</b>: Document에 대한 추가 정보를 담는 딕셔너리(dictionary) 형태의 속성입니다. 이 정보는 문서 자체의 내용이 아닌, 문서와 관련된 부가적인 데이터를 저장하는 데 사용됩니다. 위 예시에서는 {'author': '영화 해설', 'subject': '기생충 줄거리'}와 같이 저자나 주제와 같은 정보를 저장했습니다.

<b>id_</b>: 각 Document를 고유하게 식별하는 고유 ID입니다. 기본적으로는 UUID가 자동으로 생성되며, 사용자가 직접 지정할 수도 있습니다.

<b>embedding</b>: Document 텍스트를 벡터로 변환한 임베딩(embedding) 값을 저장하는 속성입니다. 이 임베딩은 텍스트의 의미를 숫자로 표현한 것으로, 검색이나 유사도 비교에 활용됩니다. 기본적으로는 비어 있으며, 임베딩 모델을 통해 생성됩니다.

excluded_embed_metadata_keys: 임베딩 생성 시 제외할 메타데이터 키 목록입니다.

excluded_llm_metadata_keys: LLM(대규모 언어 모델)이 프롬프트로 사용할 때 제외할 메타데이터 키 목록입니다.

hash: Document 내용의 변경 여부를 확인하는 데 사용되는 해시 값입니다.

metadata_template: 메타데이터를 텍스트로 변환할 때 사용할 템플릿입니다.

<b>text_template</b>: text 속성을 다른 텍스트와 결합할 때 사용할 템플릿입니다.

정리하면, metadata는 Document 객체가 가진 여러 속성 중 하나이며, text, id_, embedding 등 다른 핵심 속성들과 함께 문서를 구성하는 중요한 요소입니다.

In [29]:
document.text
document.metadata
document.id_
document.embedding

### 문서단위 검색과 결과 (분할없이)
- 실제로는 llama-index 가 내부적으로 분할한다

In [46]:
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI


# 한글 답변 설정
llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"],
							model="gpt-4o-mini", 
							system_prompt="반드시 한국어로 답변하세요.")
# Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0, system_prompt="항상 한국어로 답변하세요.")


# 텍스트 데이터를 기반으로 문서객체 생성
document_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다."
document = Document(text=document_text)

# 메타데이터 추가
document.metadata = {'author': '영화 해설', 'subject': '기생충 줄거리'}


# 문서 단위 검색을 위한 전체 문서 인덱스 생성
full_doc_index = VectorStoreIndex.from_documents([document], llm=llm)


# 검색 비교
query_text = '이 영화의 반전은 무엇인가요?'


## 문서 단위 검색
doc_query_engine = full_doc_index.as_query_engine()
doc_response = doc_query_engine.query(query_text)
print("\n문서 단위 검색 결과")
print(f"문서 검색 응답: {doc_response.response}")
if doc_response.source_nodes:
    for idx, document in enumerate(doc_response.source_nodes, start=1):
        print(f"- 결과 {idx}: {document.node.text}")


2025-08-26 15:08:02,378 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 15:08:02,849 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 15:08:03,804 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



문서 단위 검색 결과
문서 검색 응답: The movie's twist reveals the existence of a secret basement in the wealthy Park family's house, where a man has been hiding for a long time. This unexpected revelation leads to a series of events that culminate in a tragic ending.
- 결과 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.


###  SimpleNodeParser 이용 chunk

### 노드 단위 검색과 결과
- from llama_index.core.node_parser.text.sentence import SentenceSplitter
- from llama_index.core.node_parser.text.token import TokenTextSplitter
- from llama_index.core.node_parser import SemanticSplitterNodeParser

In [36]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser.text.sentence import SentenceSplitter
from llama_index.core.node_parser.text.token import TokenTextSplitter
from llama_index.core.node_parser import SemanticSplitterNodeParser

- 문장단위 (마침표 기준) chunk

### 2.3.1 SimpleNodeParser 를 이용한 텍스트 분할

In [54]:
from llama_index.core.node_parser import SimpleNodeParser

parser = SimpleNodeParser(chunk_size=80, chunk_overlap=0)
nodes = parser.get_nodes_from_documents([document])  # document는 위에서 정의

print("\n생성된 노드들")
for idx, node in enumerate(nodes, start=1):
    node.metadata = {'type': '영화 줄거리', 'genre': '드라마', 'node_id': idx}
    print(f"노드 {idx}: {node.text}")
    print(f"메타데이터: {node.metadata}")


생성된 노드들
노드 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다.
메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 1}
노드 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다.
메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 2}
노드 3: 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다.
메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 3}
노드 4: 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.
메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 4}


### 2.3.2 토큰 단위 분할

In [64]:
sample_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 그러나 이 영화의 반전은 지하실에서 시작됩니다."


token_splitter = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(sample_text)
print("=== 토큰 기반 분할 결과 ===")
for i, chunk in enumerate(token_chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")



=== 토큰 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 

Chunk 3: 쌓이며 긴장감이 점점 고조됩니다. 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



### 2.3.3 문장 단위 분할

In [65]:
from llama_index.core.node_parser.text.sentence import SentenceSplitter

splitter = SentenceSplitter(chunk_size=50, chunk_overlap=0)

sentence_chunks = splitter.split_text(sample_text)
print("=== 문장 기반 분할 결과 ===")
for i, chunk in enumerate(sentence_chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")


=== 문장 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 벌어지는 이야기입니다. 

Chunk 3: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 4: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



In [66]:
# 텍스트 데이터를 기반으로 문서객체 생성
document_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 " \
"취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, " \
"이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며," \
" 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. " \
"이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다." \
" 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다."
document = Document(text=document_text)


# NodeParser를 사용하여 문서를 작은 의미 단위(Node)로 분할
parser = SentenceSplitter(chunk_size=100, chunk_overlap=0)
nodes = parser.get_nodes_from_documents([document])



# 노드 단위 검색을 위한 인덱스 생성
llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"],
							model="gpt-4o-mini", 
							system_prompt="반드시 한국어로 답변하세요.")

node_index = VectorStoreIndex(nodes, llm=llm)

# 검색 비교
query_text = '이 영화의 반전은 무엇인가요?'


## 문장 단위 검색
print("\n문장 단위 분할 결과")
node_query_engine = node_index.as_query_engine()
node_response = node_query_engine.query(query_text)
print(f"노드 검색 응답: {node_response.response}")
if node_response.source_nodes:
    for idx, document in enumerate(node_response.source_nodes, start=1):
        print(f"- 결과 노드 {idx}: {document.node.text}")

=== 문장 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 벌어지는 이야기입니다. 

Chunk 3: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 4: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



In [ ]:
print(f"노드 검색 응답: {node_response.response}") # 한국어가 아니네..

노드 검색 응답: The movie's twist revolves around a series of events that unfold in extreme circumstances, leading to a tragic ending.


### 2.3.4 의미 단위 분할

In [59]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)

chunks = splitter.sentence_splitter(sample_text)
print("=== 의미 기반 분할 결과 ===")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")


=== 의미 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 

Chunk 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 3: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



2.3.5 허깅페이스 임베딩 이용하기
```
임베더를 로컬에 다운받아 사용. 따라서 무료이고 빠르다.
```

In [60]:
# 설치에 3분 걸림
!pip install transformers 
!pip install llama-index-embeddings-huggingface 


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-database 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.13.3 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)

chunks = splitter.sentence_splitter(sample_text)
print("=== 의미 기반 분할 결과 ===")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")

2025-08-26 15:38:26,774 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


=== 의미 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 

Chunk 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 3: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



## 2.4 인덱싱

### 2.4.2 벡터저장 인덱스

임베딩 생성은 Settings.embed_model에 지정된 모델을 통해 수행됩니다.

VectorStoreIndex는 내부적으로 Node 객체의 text를 임베딩 모델(기본적으로 OpenAI의 text-embedding-ada-002)로 벡터화합니다.

이 과정에서 각 Node의 텍스트는 벡터로 변환되어 Node.embedding 속성에 저장되며, 인덱스에 포함됩니다.

In [67]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader('data2')
documents = reader.load_data()

# Document 객체를 전달하여 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

2025-08-26 15:53:44,997 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### 2.4.3 Top-K 검색

In [68]:
# 상위 3개의 결과를 반환
query_engine = index.as_query_engine(similarity_top_k=3)

response = query_engine.query("고양이에게 수분 공급이 중요한 이유는?")

print("[검색된 상위 3개 문서]")

for idx, node in enumerate(response.source_nodes):
    print(f"\n[문서 {idx+1}]\n{node.text}")
print("\n[답변]")

print(response)

2025-08-26 15:53:58,889 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 15:53:59,794 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[검색된 상위 3개 문서]

[문서 1]
고양이는 물을 충분히 마셔야 합니다. 수분이 부족하면 신장 문제가 발생할 수 있습니다.
건식 사료보다 습식 사료가 수분 공급에 도움이 됩니다.

[문서 2]
고양이는 육식 동물입니다. 주로 고기, 생선, 그리고 가공된 고양이 사료를 먹습니다.
특히, 단백질이 풍부한 음식을 선호하며, 탄수화물 섭취는 적은 편입니다.

[문서 3]
고양이는 초콜릿, 양파, 마늘 같은 음식은 먹으면 안 됩니다.
특히, 초콜릿에 포함된 테오브로민 성분은 고양이에게 치명적일 수 있습니다.

[답변]
고양이에게 수분 공급이 중요한 이유는 신장 문제를 예방하기 위해서입니다.


## 2.5 VectorDB 

In [ ]:
# chromadb 설치 1분정도
!pip install chromadb 
!pip install llama-index-vector-stores-chroma -q

In [70]:
import chromadb

# 크로마 클라이언트 초기화
db = chromadb.PersistentClient(path="./chroma_db")

2025-08-26 15:57:51,549 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [72]:
# 컬렉션 생성하기
chroma_collection = db.get_or_create_collection("quickstart")

In [74]:
# 크로마를 벡터 저장소로 지정
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [75]:
from llama_index.core import VectorStoreIndex

# 문서 리스트 (임의의 예시)
documents = [
    Document(text="Llama2 is a large language model developed by Meta."),
    Document(text="Chroma is an open-source vector store.")
]

# 문서를 벡터 스토어 인덱스로 저장
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# 저장된 벡터 인덱스 데이터 저장
index.storage_context.persist(persist_dir="./index_data")

2025-08-26 16:00:48,399 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [76]:
# 벡터 스토어에서 저장된 인덱스 불러오기

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [ ]:
# 지금까지의 셀을 모두 하나의 셀로 
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# 크로마 클라이언트 초기화
db = chromadb.PersistentClient(path="./chroma_db")

# 저장된 컬렉션을 다시 가져오기 (또는 생성하기)
chroma_collection = db.get_or_create_collection("quickstart")

# 저장된 크로마 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 벡터 스토어 인덱스에 문서를 저장 (데이터 임베딩 후 저장)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# 인덱스 데이터를 로컬 디렉터리에 저장
index.storage_context.persist(persist_dir="./index_data")

# --- 이후 다시 데이터를 불러오고 쿼리 수행 ---
# 크로마 클라이언트 다시 초기화
db = chromadb.PersistentClient(path="./chroma_db")

# 저장된 컬렉션을 다시 가져오기
chroma_collection = db.get_or_create_collection("quickstart")

# 저장된 크로마 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 저장된 벡터 데이터를 이용해 인덱스를 다시 로드
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

# 쿼리 엔진 생성 및 쿼리 수행
query_engine = index.as_query_engine()
response = query_engine.query("What is Chroma?")
print(response)

Chroma is an open-source vector store.


## 2.6 쿼리

### 2.6.1 쿼리 엔진

In [77]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "고객의 개인 정보를 참고하여 맞춤형 이메일을 작성해 주세요."
)
print(response)

2025-08-26 16:06:56,934 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 16:06:57,498 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I'm sorry, I cannot assist with that request.


### 2.6.2 검색

In [78]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5, # 상위 5개의 결과 반환
)

### 2.6.3 후처리

In [79]:
from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)
query_engine = index.as_query_engine(node_postprocessors=[postprocessor])

### 2.6.4 응답 합성

In [80]:
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# 응답 합성기 설정
response_synthesizer = get_response_synthesizer(response_mode="compact")

# 쿼리 엔진 구성
query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(),
    response_synthesizer=response_synthesizer,
)

# 쿼리 실행
response = query_engine.query("Llama2란?")

2025-08-26 16:08:14,552 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 16:08:15,214 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### 2.6.5 커스터마이징

In [81]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# 문서를 기반으로 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

2025-08-26 16:08:23,560 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [82]:
# 검색기 설정 (상위 10개의 유사한 결과 반환)
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

In [83]:
# 응답 합성기 설정
response_synthesizer = get_response_synthesizer()

In [84]:
# 후처리 설정 (유사도 0.7 이상인 노드만 선택)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)

In [85]:
# 쿼리 엔진
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [86]:
# 쿼리 실행 및 결과 출력
response = query_engine.query("모나리자 그림은 어디에 전시되어 있나요?")
print(response)

2025-08-26 16:08:30,827 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 16:08:31,364 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I'm sorry, I cannot provide an answer to that question based on the context information provided.
